In [2]:
#Run these beforehand
library(tidyverse)
library(repr)
library(tidymodels)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

# Data Description
The dataset is separated into two lists. One is a list of 196 unique players with 7 variables of data on each player:
- experience, their experience as one of 'Beginner', 'Regular', 'Amateur', 'Veteran', or 'Pro' (chr)
- hashedEmail, Private hashed email address (chr)
- name, player name (chr)
- gender, player gender (chr)
- played_hours, number of hours they have played on the server (dbl)
- Age, player age in years (dbl)
- subscribe, whether or not they are subscribed to a games-related news letter (lgl)

Some issues I noticed with this list was that experience hierarchy is unclear (such as wehther or not 'Regular' is more experienced than 'Amatuer'), there are NA values in the age, and that it is unclear which news letter the players are subscribed to.

The other is a list of individual play sessions with 5 variables on each session:
- hashedEmail, private hashed email address (chr)
- start_time, start time of session in day/month/year time (chr)
- end_time, end time of session in day/month/year time (chr)
- original_start_time, Unix epoch start time in milliseconds (dbl)
- original_end_time, Unix epoch end time in milliseconds (dbl)

Some issues I noticed with this list is that the start and end include the time and date in the same column, and should probably be seperated. It also lacks the session time length, which is probably the most basic piece of data which should be included as it's own column. I am also not certain if every session is associated with a hashedEmail in the player list. 

# Questions
For this project I would like to focus on Question 2, or what "kinds" of players are most likely to contribute a large amount of data so that we can target those players in our recruiting efforts. The specific question I would like to ask is: Can player experience and and age predict the length of play time in the list of sessions for that player. To answer this question, I would use the data to determine if age and total session length have a linear assocation. I could also either split the data by the categories of experience, or use k-nn fit algorithms to see which creates a better prediction system. Since there are NA values in the age, I would likely have to remove those observations in the cases when examining age, and I would also likely need to create a new session length column in the list of sessions for ease of use. 

In [3]:
players <- read_csv("players.csv")
sessions <- read_csv("sessions.csv")

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [5]:
clean_sessions <- separate(sessions, start_time, c("start_date", "start_time"), " ") |>
    separate(end_time, c("end_date", "end_time"), " ")
colnames(clean_sessions) <- c("hashed_email", "start_date", "start_time", "end_date", "end_time", "original_start_time", "original_end_time")
head(clean_sessions)

hashed_email,start_date,start_time,end_date,end_time,original_start_time,original_end_time
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,30/06/2024,18:12,30/06/2024,18:24,1.71977e+12,1.71977e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,17/06/2024,23:33,17/06/2024,23:46,1.71867e+12,1.71867e+12
f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc,25/07/2024,17:34,25/07/2024,17:57,1.72193e+12,1.72193e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,25/07/2024,03:22,25/07/2024,03:58,1.72188e+12,1.72188e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,25/05/2024,16:01,25/05/2024,16:12,1.71665e+12,1.71665e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,23/06/2024,15:08,23/06/2024,17:10,1.71916e+12,1.71916e+12
